In [3]:

import sys
sys.path.append('/home/nick/work/MLIR/build/tools/mlir/python_packages/mlir_core')
sys.path.append('/home/nick/work/mlir_nelli')

import numpy as np

import mlir.extras.types as T
from mlir.extras.ast.canonicalize import canonicalize
from mlir.extras.context import ContextManager
from mlir.extras.dialects.ext.arith import constant
from mlir.extras.dialects.ext.memref import S
from mlir.extras.dialects.ext.func import toMLIRFunc
from mlir.extras.dialects.ext.scf import canonicalizer as scf_canonicalizer, range_ as range
from mlir.extras.runtime.passes import Pipeline, run_pipeline
from mlir.extras.runtime.refbackend import LLVMJITBackend
from mlir.ir import StridedLayoutAttr

# create MLIR context
CM = ContextManager()
Ctx = CM.__enter__()
backend = LLVMJITBackend()

K = 10
memref_i64 = T.memref(K, K, T.i64())

@toMLIRFunc(emit=True)
@canonicalize(using=scf_canonicalizer) # AST transformer and Bytecode patcher
def memfoo(A: memref_i64, B: memref_i64, C: memref_i64):
    one = constant(1)
    two = constant(2)
    if one > two:
        C[0, 0] = constant(3, T.i64())
    else:
        for i in range(0, K):
            for j in range(0, K):
                C[i, j] = A[i, j] * B[i, j]



run_pipeline(Ctx.module, Pipeline().cse())
print(Ctx.module)
CM.__exit__(None, None, None)


module {
  func.func @memfoo(%arg0: memref<10x10xi64>, %arg1: memref<10x10xi64>, %arg2: memref<10x10xi64>) {
    %c1_i32 = arith.constant 1 : i32
    %c2_i32 = arith.constant 2 : i32
    %0 = arith.cmpi ugt, %c1_i32, %c2_i32 : i32
    scf.if %0 {
      %c3_i64 = arith.constant 3 : i64
      %c0 = arith.constant 0 : index
      %c0_0 = arith.constant 0 : index
      memref.store %c3_i64, %arg2[%c0, %c0_0] : memref<10x10xi64>
    } else {
      %c0 = arith.constant 0 : index
      %c10 = arith.constant 10 : index
      %c1 = arith.constant 1 : index
      scf.for %arg3 = %c0 to %c10 step %c1 {
        %c0_0 = arith.constant 0 : index
        %c10_1 = arith.constant 10 : index
        %c1_2 = arith.constant 1 : index
        scf.for %arg4 = %c0_0 to %c10_1 step %c1_2 {
          %1 = memref.load %arg0[%arg3, %arg4] : memref<10x10xi64>
          %2 = memref.load %arg1[%arg3, %arg4] : memref<10x10xi64>
          %3 = arith.muli %1, %2 : i64
          memref.store %3, %arg2[%arg3, %arg4] : m

False